In [1]:
# Francisco Dominguez Mateos
# 25/08/2021
# Voice Stress detection

In [2]:
import glob
import time
import numpy as np
#np.set_printoptions(precision=3)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [3]:
path_base="/home/francisco/datasets/datasets/sound/voice/stress"

In [4]:
def getLabel(s):
    if s=="TRUE":
        return 1.0
    if s=="FALSE":
        return 0.0
    if s=="PC":
        return 2.0
def isRightLabel(s):
    if s=="TRUE":
        return True
    if s=="FALSE":
        return True
    if s=="PC":
        return False
    return False

In [5]:
data=[]
labels=[]
file_txt=path_base+"/*.csv"
for filepath in glob.glob(file_txt):
    print("Processing file: {}".format(filepath)) 
    with open(filepath) as fp:  
        line = fp.readline()
        head_list=line.split(";")
        #for i,head in enumerate(head_list):
        #    print(i,head)
        cnt = 1
        line = fp.readline()
        while line:
            data_row=[]
            line_list=line.split(";")
            #print("Line {}: {} {} {}".format(cnt, line_list[0], line_list[1], line_list[2]))
            line = fp.readline()
            cnt += 1
            if not isRightLabel(line_list[0]):
                continue
            for i,datum in enumerate(line_list):
                #print(i,head_list[i],"=",datum)
                if i>1:
                    data_row.append(float(datum))
                if i==0:
                    labels.append(getLabel(datum))
            data.append(data_row)
data_np=np.array(data)
labels_np=np.array(labels)
print(data_np.shape)
print(data_np)
print(labels_np.shape)
print(labels_np)

Processing file: /home/francisco/datasets/datasets/sound/voice/stress/Set_Males.csv
Processing file: /home/francisco/datasets/datasets/sound/voice/stress/Set_Females.csv
(50, 68)
[[126.278 0.014 0.029 ... 15.681 0.870 0.012]
 [142.901 0.005 0.019 ... 10.834 0.936 0.008]
 [117.189 0.005 0.014 ... 11.765 0.967 0.006]
 ...
 [192.269 0.008 0.022 ... 10.600 0.928 0.007]
 [167.700 0.005 0.025 ... 9.564 0.933 0.009]
 [206.086 0.007 0.013 ... 9.851 0.922 0.046]]
(50,)
[1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000]


In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
ss=StandardScaler()
data_ss=ss.fit_transform(data_np)

print(data_ss.shape)
print(data_ss)

(50, 68)
[[-0.692 1.335 0.942 ... 1.056 -1.550 -0.524]
 [-0.309 -0.907 -0.100 ... -0.261 0.011 -0.808]
 [-0.901 -0.907 -0.621 ... -0.008 0.744 -0.950]
 ...
 [0.826 -0.159 0.213 ... -0.325 -0.178 -0.879]
 [0.261 -0.907 0.525 ... -0.606 -0.060 -0.737]
 [1.144 -0.409 -0.725 ... -0.528 -0.320 1.895]]


In [8]:
from sklearn.decomposition import PCA

In [9]:
#pca=PCA(7,whiten=True)
#data_ss=pca.fit_transform(data_np)
#print(data_ss.shape)

In [10]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis

In [11]:
random_state = 0
X=data_ss
y=labels_np
# Reduce dimension to 2 with NeighborhoodComponentAnalysis
nca =NeighborhoodComponentsAnalysis(n_components=2,random_state=random_state)
data_ss=nca.fit(X, y).transform(X)
print(data_ss.shape)

(50, 2)


In [12]:
import numpy as onp
import jax.numpy as np
from jax import grad, jit, vmap, value_and_grad
from jax import random
from jax.experimental import stax
from jax.experimental.stax import (BatchNorm, Conv, Dense, Flatten, Dropout,
                                   Relu, LogSoftmax)

In [13]:
# Import some additional JAX and dataloader helpers
from jax.scipy.special import logsumexp
from jax.experimental import optimizers

In [14]:
#Test if JAX is using CPU or GPU
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [15]:
# Generate key which is used to generate random numbers
key = random.PRNGKey(1)

In [16]:
num_classes = 2
hidden=10
dropout_rate=0.5
def makeNet(num_classes,hidden,dropout_rate,mode="train"):
    init_fun, net = stax.serial(
        Dense(hidden),
        #BatchNorm(axis=0),
        Relu,
        #Dropout(dropout_rate,mode=mode),
        #Dense(hidden),
        #BatchNorm(axis=0),
        #Relu,
        #Dropout(dropout_rate,mode=mode),
        #Dense(hidden),
        Dense(num_classes),
        LogSoftmax)
    return init_fun,net

In [17]:
def initNets(data):
    #buid net
    init_fun, net=makeNet(num_classes,hidden,dropout_rate)
    input_shape=(-1,)+ data.shape[1:]
    output_shape, params = init_fun(key, input_shape)
    #print("ouput_shape=",output_shape)
    _,netTest=makeNet(num_classes,hidden,dropout_rate,mode='test')  
    #buid optimizer
    step_size = 1e-4
    opt_init, opt_update, get_params = optimizers.adam(step_size)
    opt_state = opt_init(params)
    return net,netTest,opt_state,get_params,opt_update

In [18]:
net,netTest,opt_state,get_params,opt_update=initNets(data_ss)

In [19]:
from jax.tree_util import tree_flatten

def l2_squared(pytree):
  leaves, _ = tree_flatten(pytree)
  return sum(np.vdot(x, x) for x in leaves)

In [20]:
def one_hot(x, k, dtype=np.float32):
    """Create a one-hot encoding of x of size k """
    return np.array(x[:, None] == np.arange(k), dtype)

def loss(params, data, targets,key):
    preds = net(params, data,rng=key)
    return -np.sum(preds * targets)+l2_squared(params)

def predict(params,data,key):
    preds=netTest(params,data,rng=key)
    return np.exp(preds)

def accuracy(p,targets):
    target_class    = np.argmax(targets, axis=1)
    predicted_class = np.argmax(p      , axis=1)
    acc_total       = np.sum(predicted_class == target_class)
    return acc_total/p.shape[0]


In [21]:
@jit
def update(params, x, y, opt_state,key):
    """ Compute the gradient for a batch and update the parameters """
    value, grads = value_and_grad(loss)(params, x, y,key)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

In [22]:
labels_onehot=one_hot(labels_np,num_classes)
print(labels_onehot.shape)
def run_training_loop(data_ss,labels_onehot,num_epochs, opt_state,key,verbose=False):
    """ Implements a learning loop over epochs. """
    # Get the initial set of parameters
    params = get_params(opt_state)

    # Loop over the training epochs
    for epoch in range(num_epochs):
        rng,key=random.split(key)
        start_time = time.time()
        params, opt_state, loss = update(params, data_ss, labels_onehot, opt_state,rng)
        epoch_time = time.time() - start_time
        if verbose: print("Epoch {} | T: {:0.2f} | loss: {:0.3f} ".format(epoch+1, epoch_time,
                                                                    loss))
    return loss,params

(50, 2)


In [23]:
net,netTest,opt_state,get_params,opt_update=initNets(data_ss)
l,params=run_training_loop(data_ss,labels_onehot,2000,opt_state,key,False)
print(l)

50.53074


In [24]:
def _numpy_delete(x, idx):
    """
    Gets the subarray from `x` where data from index `idx` on the first axis is removed.
    """
    # NB: numpy.delete is not yet available in JAX
    mask = np.arange(x.shape[0] - 1) < idx
    return np.where(mask.reshape((-1,) + (1,) * (x.ndim - 1)), x[:-1], x[1:])
def allButOne(x,y,i):
    global key
    oneX=x[i:i+1] #this return a (1,N) shape vs x[i] gives (N,) shape
    oneY=y[i:i+1]
    allXBut=_numpy_delete(x,i)
    allYBut=_numpy_delete(y,i)
    rng,key=random.split(key)
    idxs=allXBut.shape[0]
    suffleIdx=random.permutation(rng,idxs)
    allXBut=allXBut[suffleIdx]
    allYBut=allYBut[suffleIdx]
    return allXBut,allYBut,oneX,oneY

In [25]:
allXBut,allYBut,oneX,oneY=allButOne(data_ss,labels_onehot,1)
print(allXBut.shape)
print(allYBut.shape)
print(oneX.shape)
print(oneY.shape)
oneY

(49, 2)
(49, 2)
(1, 2)
(1, 2)


DeviceArray([[0.000, 1.000]], dtype=float32)

In [26]:
def runLeaveOneOut(data,labels_onehot,epochs=2000,verbose=True):
    accT=0
    for i in range(data_ss.shape[0]):
        allXBut,allYBut,oneX,oneY=allButOne(data,labels_onehot,i)
        loss,params=run_training_loop(allXBut,allYBut,epochs,opt_state,key)
        p=predict(params,oneX,key)
        acc=accuracy(p,oneY)
        accT+=acc
        if verbose:
            print("i=",i,p,oneY,"acc={0:0.3f} accMean={1:0.3f} loss={2:0.3f}".format(acc,accT/(i+1),loss))
    return accT/(data_ss.shape[0])

In [27]:
accMean=runLeaveOneOut(data_ss,labels_onehot,epochs=10000)
print("accMean={0:0.3f}".format(accMean))

i= 0 [[0.551 0.449]] [[0.000 1.000]] acc=0.000 accMean=0.000 loss=32.403
i= 1 [[0.350 0.650]] [[0.000 1.000]] acc=1.000 accMean=0.500 loss=32.806
i= 2 [[0.594 0.406]] [[0.000 1.000]] acc=0.000 accMean=0.333 loss=32.170
i= 3 [[0.578 0.422]] [[0.000 1.000]] acc=0.000 accMean=0.250 loss=32.227
i= 4 [[0.814 0.186]] [[0.000 1.000]] acc=0.000 accMean=0.200 loss=31.667
i= 5 [[0.540 0.460]] [[0.000 1.000]] acc=0.000 accMean=0.167 loss=32.453
i= 6 [[0.483 0.517]] [[0.000 1.000]] acc=1.000 accMean=0.286 loss=32.659
i= 7 [[0.245 0.755]] [[0.000 1.000]] acc=1.000 accMean=0.375 loss=32.945
i= 8 [[0.443 0.557]] [[0.000 1.000]] acc=1.000 accMean=0.444 loss=32.723
i= 9 [[0.183 0.817]] [[0.000 1.000]] acc=1.000 accMean=0.500 loss=33.014


KeyboardInterrupt: 

In [99]:
for i in range(1,data_np.shape[0]):
    #pca=PCA(i,whiten=True)
    #data_ss=pca.fit_transform(data_np)
    nca =NeighborhoodComponentsAnalysis(n_components=2,random_state=random_state)
    data_ss=nca.fit(X, y).transform(X)
    print(pca.explained_variance_ratio_)
    print(data_ss.shape)
    net,netTest,opt_state,get_params,opt_update=initNets(data_ss)
    accMean=runLeaveOneOut(data_ss,labels_onehot,epochs=10000,verbose=False)
    print("i={0:} accMean={1:0.3f}".format(i,accMean))

[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=1 accMean=0.440
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=2 accMean=0.440
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=3 accMean=0.620
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=4 accMean=0.540
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=5 accMean=0.600
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=6 accMean=0.620
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=7 accMean=0.480
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=8 accMean=0.400
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=9 accMean=0.700
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=10 accMean=0.500
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=11 accMean=0.540
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=12 accMean=0.600
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=13 accMean=0.580
[0.918 0.076 0.005 0.001 0.000 0.000 0.000]
(50, 2)
i=14 accMean=0.600
[0.918 0.076 0.